In [1]:
import pandas as pd
import re
import html
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from bs4 import BeautifulSoup
from tqdm import tqdm
import math
import preprocessor as p
tqdm.pandas(desc="df")
stop_words = set(stopwords.words('english'))

In [2]:
contractions = { 
"ain't": "is not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he had",
"he'd've": "he would have",
"he'll": "he will",
"he'll've": "he will have",
"he's": "he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how is",
"I'd": "I had",
"I'd've": "I would have",
"I'll": "I will",
"I'll've": "I will have",
"I'm": "I am",
"I've": "I have",
"isn't": "is not",
"it'd": "it would",
"it'd've": "it would have",
"it'll": "it will",
"it'll've": "it will have",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she would",
"she'd've": "she would have",
"she'll": "she will",
"she'll've": "she will have",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so as / so is",
"that'd": "that would",
"that'd've": "that would have",
"that's": "that is",
"there'd": "there would",
"there'd've": "there would have",
"there's": "there is",
"they'd": "they would",
"they'd've": "they would have",
"they'll": "they will",
"they'll've": "they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what'll've": "what will have",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"when's": "when is",
"when've": "when have",
"where'd": "where did",
"where's": "where is",
"where've": "where have",
"who'll": "who will",
"who'll've": "who will have",
"who's": "who is",
"who've": "who have",
"why's": "why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you would",
"you'd've": "you would have",
"you'll": "you will",
"you'll've": "you will have",
"you're": "you are",
"you've": "you have"
}

In [3]:
original_df = pd.read_csv('all.csv')

C:\Users\dhruv\AppData\Roaming\Python\Python38\site-packages\IPython\core\interactiveshell.py:3012: DtypeWarning: Columns (0,1,2,3,4) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
#remove null values
df = original_df[original_df['author'].notna()].copy()
df.sort_values(by=["author"])

,Unnamed: 0,flair_text,body,subreddit,author
18255377,11901394,ISTP: The Analyzer,Big Brother is watching. Be safe gentlemen!,Cyberpunk,---MrTONY---
8063066,5367718,ISTP: The Analyzer,Which song is that from?,LilPeep,---MrTONY---
34642465,24157148,ISTP: The Analyzer,Lel,drunk,---MrTONY---
9900636,6489766,ISTP: The Analyzer,I automatically down vote shit posts like this,drunk,---MrTONY---
15538225,10400684,ISTP: The Analyzer,Did u watch it on drugs?,batman,---MrTONY---
...,...,...,...,...,...
15290865,10153339,INTJ: The Mastermind,It's called a swap file. It's the opposite of...,buildapc,zylo_
35184181,24615797,INTJ: The Mastermind,The whole point of food stamps is to provide n...,facepalm,zylo_
38310080,26093346,INTJ: The Mastermind,I think the argument is that he was scared onc...,politics,zylo_
17821776,11835412,INTJ: The Mastermind,"I assure you, they exist.",AskReddit,zylo_


In [5]:
#extract all MBTI values per author
df['MBTI'] = df.progress_apply(lambda row: re.search("[IiEe][SsNn][FfTt][PpJj]",row.flair_text)[0].upper(), axis=1)
df['MBTIs'] = df.progress_apply(lambda row: re.findall("[IiEe][SsNn][FfTt][PpJj]",row.flair_text), axis=1)
df

df: 100%|███████████████████████████████████████████████████████████████| 27988820/27988820 [11:01<00:00, 42308.36it/s]


,Unnamed: 0,flair_text,body,subreddit,author,MBTI,MBTIs
0,0,INFP: The Dreamer Senpai,Lol that's why I left.,entp,LadyBanterbury,INFP,[INFP]
1,1,INTP: The Theorist,"I was just about to post ""I try telling people...",INTP,Finarin,INTP,[INTP]
2,2,INFP: The Dreamer,My first thought was Pepsi or something. Proba...,WTF,xanplease,INFP,[INFP]
3,3,"ENTP: Antisocial Extrovert, Rational Eccentric","Not if the formula is something like ""every ti...",youtubehaiku,HeirToGallifrey,ENTP,[ENTP]
4,4,INTP/18/m/blankly staring at you,Does this imply I'm a five now?,entp,lightfive,INTP,[INTP]
...,...,...,...,...,...,...,...
40774593,27991297,INFP: The Dreamer,Sounds like being in a heathy relationship wit...,infp,LazerKittenz,INFP,[INFP]
40774594,27991298,INTP/53115 of 5,"I did a search for ""Communist nation HAHAHAHAH...",vexillology,supremecrafters,INTP,[INTP]
40774595,27991299,INFJ: The Protector,same meaning.,Surface,sinwarrior,INFJ,[INFJ]
40774596,27991300,26 &amp; entp. but what's my enneagram?,"Sure, I'll be glad to, though I'd appreciate i...",changemyview,OurSuiGeneris,ENTP,[entp]


In [6]:
#remove all authors who have multiple MBTI personalities
author_to_remove = df[df['MBTIs'].str.len() > 1]['author'].tolist()
df1 = df[~df['author'].isin(author_to_remove)].copy()
author_to_remove

['RockinSocksReborn',
 'Killerzeit',
 'owlsymbolism',
 '2_4_16_256',
 'Djzongreethesecond',
 'theCroc',
 'LjSpike',
 'Milswanca69',
 'aim2free',
 'DMPDrugs',
 'nut_conspiracy_nut',
 'MonsieurPhilippe',
 'Spearka',
 'lurkerino95',
 'captain_zavec',
 'TMA-3',
 'PoeDancer',
 'nut_conspiracy_nut',
 'emceecombs',
 '-Afrodisiac-',
 'sgst',
 'Jayy-',
 'DMPDrugs',
 'daaaammn',
 'Vyrmayn_Gothryn',
 'bohr12',
 'omegaxis',
 'dejoblue',
 'dHUMANb',
 'justifications',
 'Redditis4virgins',
 'AlenaBrolxFlami',
 'TatianaAlena',
 'TatianaAlena',
 'The_White_Baron',
 'pushka',
 'AlenaBrolxFlami',
 'DOOMSPECTRE',
 'mistertime',
 'millenmar',
 'AlenaBrolxFlami',
 'yabluko',
 'ScalSaver',
 'lightfive',
 'lancerusso',
 'shtzkrieg',
 'Draco309',
 'emceecombs',
 'MortRouge',
 'Scarcer',
 'nut_conspiracy_nut',
 'WittyOriginalName',
 'shtzkrieg',
 'evangelion-zero-one',
 'Spearka',
 'hopewings',
 'aim2free',
 'TMA-3',
 'Vyrmayn_Gothryn',
 'is-this-kosher',
 'theCroc',
 'BoxOfDust',
 'owlsymbolism',
 'TMA-3',
 '

In [8]:
len(df1)

24609192

In [9]:
#drop all null values and extra columns
df1.drop(columns=['flair_text','subreddit','MBTIs','author'], inplace=True)
df1 = df1.dropna()
df1

,Unnamed: 0,body,MBTI
0,0,Lol that's why I left.,INFP
1,1,"I was just about to post ""I try telling people...",INTP
2,2,My first thought was Pepsi or something. Proba...,INFP
3,3,"Not if the formula is something like ""every ti...",ENTP
5,5,"Well, I wouldn't know but I think there's a lo...",INTP
...,...,...,...
40774593,27991297,Sounds like being in a heathy relationship wit...,INFP
40774594,27991298,"I did a search for ""Communist nation HAHAHAHAH...",INTP
40774595,27991299,same meaning.,INFJ
40774596,27991300,"Sure, I'll be glad to, though I'd appreciate i...",ENTP


In [13]:
#narrow down dataframe to comments ranging between 20 to 50 words
short_df = df1[df1['body'].str.count(' ').between(20,50)].copy()
short_df

,Unnamed: 0,body,MBTI
1,1,"I was just about to post ""I try telling people...",INTP
13,13,"maybe you were being tongue in cheek, but you ...",ENTP
16,16,Its kind of strange how they will lump very va...,INTJ
17,17,"Wow, that's some fucked up woman. I'm kind of ...",INFP
22,22,I've never been to this subreddit and I ain't ...,INFP
...,...,...,...
40774587,27991291,The 1st ~~third~~ half of The Phantom Pain is ...,INFP
40774590,27991294,Yeah...\n\nSometimes I just decide to let loos...,INFP
40774591,27991295,Sounds like maybe the Eric Johnson Dunlop Jazz...,INFP
40774594,27991298,"I did a search for ""Communist nation HAHAHAHAH...",INTP


In [14]:
def preprocess_text(sentence):
    #clean sentence using built in preprocessors
    sentence = str(sentence)
    sentence = p.clean(sentence)
    #tokenize word and remove stop words
    word_tokens = word_tokenize(sentence)
    sentence = [w for w in word_tokens if not w.lower() in stop_words]
    sentence = [x for x in sentence if "'" not in x]
    sentence = ' '.join(sentence)
    # Removing multiple spaces and line breaks
    sentence = re.sub(r"\s+", " ", sentence)
    sentence = sentence.strip()
    sentence = sentence.replace('\n',' ')
    return sentence

In [15]:
short_df['clean'] = short_df['body'].progress_apply(lambda x:preprocess_text(x))

df: 100%|██████████████████████████████████████████████████████████████████| 6483257/6483257 [42:17<00:00, 2554.69it/s]


In [16]:
short_df['clean'].head().tolist()

['post `` try telling people time always joking unless say serious right still catch ! already figured secret . like know .',
 'maybe tongue cheek , wreck body enjoy using drugs . eat/sleep well stay hydrated . vitamins ca hurt either',
 'kind strange lump vague references religions onto things make explicit . would interesting aesthetic closely match . characters , religions , etc .',
 'Wow , fucked woman . kind worried child , ( unlike fianc ) ca distance toxic person .',
 'never subreddit ai even british drink enough tea know exactly talking']

In [17]:
short_df['body'].head().tolist()

['I was just about to post "I try telling people all the time that I\'m always joking unless I say \'I am being serious right now\' and they still don\'t catch on!"  But here you\'ve already figured out the secret.  It\'s like you know me.',
 "maybe you were being tongue in cheek, but you don't have to wreck your body to enjoy using drugs here and there. eat/sleep well and stay hydrated. some vitamins can't hurt either",
 'Its kind of strange how they will lump very vague references to other religions onto things but not make it more explicit that they are doing this. It would be more interesting if they had an aesthetic to more closely match. If not on the characters, then the religions, etc.',
 "Wow, that's some fucked up woman. I'm kind of worried for her child, who (unlike your fiancé) can't distance himself from such a toxic person. ",
 "I've never been to this subreddit and I ain't even british but I drink enough tea to know exactly what you're talking about "]

In [18]:
#narrow down dataframe again to comments with 20-30 words
short_df2 = short_df[short_df['clean'].str.count(' ').between(20,30)].copy()

In [19]:
short_df2['numConcat'] = 1
short_df2

,Unnamed: 0,body,MBTI,clean,numConcat
1,1,"I was just about to post ""I try telling people...",INTP,post `` try telling people time always joking ...,1
16,16,Its kind of strange how they will lump very va...,INTJ,kind strange lump vague references religions o...,1
27,27,I helped some friends of mine purchase two Hon...,INTP,helped friends mine purchase two Honda Civic S...,1
32,32,"I do too, but can one come back from it all? H...",INTJ,", one come back ? , indeed , need meds , chang...",1
41,41,He promised to fuck the climate in his campaig...,ISTP,promised fuck climate campaign . promises fuck...,1
...,...,...,...,...,...
40774560,27991264,"There's no problem with meanness lol, we don't...",INTP,"problem meanness lol , play Rust hugs whatever...",1
40774572,27991276,&gt;Torture is wrong because it is ineffective...,ENTP,& gt ; Torture wrong ineffective.That incohere...,1
40774578,27991282,But now the question stands. What actions must...,ENTP,question stands . actions must take get shit t...,1
40774582,27991286,"Given your columns line up with what is shown,...",INTJ,"Given columns line shown , use formula like : ...",1


In [20]:
allCounts = short_df2['MBTI'].value_counts()
print(allCounts)
print(min(allCounts))
allCountsDic = allCounts.to_dict()
minKey = min(allCountsDic, key=allCountsDic.get)
print(minKey)

INFP    526563
INFJ    459903
INTP    452147
ENTP    320714
INTJ    270995
ENTJ    110590
ISTP    104345
ENFP     83392
ENFJ     53969
ISFJ     19837
ESTP     19296
ISTJ     18075
ISFP     15840
ESTJ     11096
ESFP      7725
ESFJ      6717
Name: MBTI, dtype: int64
6717
ESFJ


In [21]:
short_df2['E/I'] = short_df2.progress_apply(lambda row: row.MBTI[0], axis=1)
short_df2['N/S'] = short_df2.progress_apply(lambda row: row.MBTI[1], axis=1)
short_df2['F/T'] = short_df2.progress_apply(lambda row: row.MBTI[2], axis=1)
short_df2['P/J'] = short_df2.progress_apply(lambda row: row.MBTI[3], axis=1)

df: 100%|█████████████████████████████████████████████████████████████████| 2481204/2481204 [00:28<00:00, 85861.82it/s]


In [22]:
#downscale classes to get similar distributions
distrib = dict(allCounts)

counts = {}
for i in distrib:
    for j in i:
        c = counts.get(j,0)
        counts[j] = c+distrib[i]
ie = counts["I"] - counts["E"]

total = distrib["INTP"] + distrib["INFP"] + distrib["INTJ"] + distrib["INFJ"] +distrib["ISTP"] + distrib["ISFP"] + distrib["ISTJ"] + distrib["ISFJ"]
for i in distrib:
    if i[0] == "I":
        toremove = int(ie*(distrib[i]/total))
        distrib[i] -= toremove
        

counts = {}
for i in distrib:
    for j in i:
        c = counts.get(j,0)
        counts[j] = c+distrib[i]
ns = counts["N"] - counts["S"]

total = distrib["INTP"] + distrib["INFP"] + distrib["INTJ"] + distrib["INFJ"]+ distrib["ENTP"] + distrib["ENFP"] + distrib["ENTJ"] + distrib["ENFJ"] 
for i in distrib:
    if i[1] == "N":
        toremove = int(ns*(distrib[i]/total))
        distrib[i] -= toremove

        
counts = {}
for i in distrib:
    for j in i:
        c = counts.get(j,0)
        counts[j] = c+distrib[i]

ft = counts["T"] - counts["F"]
total = distrib["ENTP"] + distrib["INTP"] + distrib["INTJ"] + distrib["ENTJ"] +distrib["ISTP"] + distrib["ESTP"] + distrib["ISTJ"] + distrib["ESTJ"]
for i in distrib:
    if i[2] == "T":
        toremove = int(ft*(distrib[i]/total))
        distrib[i] -= toremove
        

counts = {}
for i in distrib:
    for j in i:
        c = counts.get(j,0)
        counts[j] = c+distrib[i]

In [23]:
distrib

{'INFP': 14809,
 'INFJ': 12935,
 'INTP': 6531,
 'ENTP': 14102,
 'INTJ': 3915,
 'ENTJ': 4863,
 'ISTP': 17602,
 'ENFP': 7140,
 'ENFJ': 4621,
 'ISFJ': 6517,
 'ESTP': 9910,
 'ISTJ': 3050,
 'ISFP': 5204,
 'ESTJ': 5699,
 'ESFP': 7725,
 'ESFJ': 6717}

In [24]:
total = 0
for i in distrib: 
    total += distrib[i]

print("Need to add:", total)
entries = []
for index, row in tqdm(short_df2.iterrows()):
    if distrib[row['MBTI']] > 0:
        entries.append(list(row))
        distrib[row['MBTI']] -= 1
        total -= 1
    if total == 0:
        break
        
df5 = pd.DataFrame(entries)
df5.columns = list(short_df2.columns)


0it [00:00, ?it/s]

Need to add: 131340


2481103it [02:28, 16750.19it/s]


In [25]:
final = df5[['MBTI','clean','numConcat']].copy()
final = final.reset_index().reset_index()
final = final.rename(columns={'MBTI':'type','clean':'text','numConcat':'split','level_0':'id'})
final = final[['type','text','split','id']]
final

,type,text,split,id
0,INTP,post `` try telling people time always joking ...,1,0
1,INTJ,kind strange lump vague references religions o...,1,1
2,INTP,helped friends mine purchase two Honda Civic S...,1,2
3,INTJ,", one come back ? , indeed , need meds , chang...",1,3
4,ISTP,promised fuck climate campaign . promises fuck...,1,4
...,...,...,...,...
131335,ESFJ,EKIN . forget exact number . class small fills...,1,131335
131336,ESFJ,"Congrats ! ! senior , makes nostalgic sad , ex...",1,131336
131337,ESFP,"Oh called . sanguine choleric * , whatever mea...",1,131337
131338,ESFJ,traditional calendar ; Extraordinary Form pari...,1,131338


In [26]:
c = dict(final["type"].value_counts())
d = {}
for i in c:
    for j in i:
        co = d.get(j,0)
        d[j] = co +c[i]
d

{'I': 70563,
 'S': 62424,
 'T': 65672,
 'P': 83023,
 'N': 68916,
 'F': 65668,
 'E': 60777,
 'J': 48317}

In [27]:
final.to_csv('17mildata_cleaned.csv',index=False)